In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import random
import shutil

In [54]:
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import InputLayer,Conv2D,MaxPool2D,Dense,Flatten,Dropout,concatenate,BatchNormalization
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import image
from keras.applications.vgg16 import preprocess_input,decode_predictions
from keras.utils import np_utils

## Classical CNN

## preprocessing

In [57]:
from keras.datasets import mnist

In [58]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()

In [59]:
x_train.shape

(60000, 28, 28)

In [60]:
x_train = x_train.reshape(len(x_train),x_train.shape[1],x_train.shape[2],1)
x_test = x_test.reshape(len(x_test),x_test.shape[1],x_test.shape[2],1)

In [61]:
x_train= x_train/255.0
x_test = x_test/255.0

In [ ]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

## Building model

In [94]:
model = Sequential()

model.add(Conv2D(filters=32 , kernel_size=3 , activation = 'relu' ,input_shape=x_train[0].shape))
model.add(MaxPool2D(pool_size=2,strides=2))

model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(units=512,activation='relu'))
model.add(Dense(units=10 , activation='softmax'))

In [95]:
model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_60 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
batch_normalization_21 (Batc (None, 13, 13, 32)        128       
_________________________________________________________________
flatten_8 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 512)               2769408   
_________________________________________________________________
dense_17 (Dense)             (None, 10)                5130      
Total params: 2,774,986
Trainable params: 2,774,922
Non-trainable params: 64
__________________________________________

In [96]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['acc'])

In [97]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=32,epochs=2)

Epoch 1/2
1875/1875 [==============================] - 88s 47ms/step - loss: 0.1099 - acc: 0.9668 - val_loss: 0.0717 - val_acc: 0.9804
Epoch 2/2
1875/1875 [==============================] - 79s 42ms/step - loss: 0.0382 - acc: 0.9879 - val_loss: 0.0684 - val_acc: 0.9803


## Inception Modules

In [127]:
input_img = Input(shape =(x_train[0].shape))

tower1    = Conv2D(filters=32,kernel_size=1,activation='relu',padding='same')(input_img)
tower1 = Conv2D(filters = 32 , kernel_size = 3 , padding ='same' ,activation = 'relu')(tower1)

tower2 = Conv2D(filters = 32 , kernel_size =1 , padding='same' , activation = 'relu')(input_img)
tower2 = Conv2D(filters = 32 , kernel_size = 5 , padding='same' , activation = 'relu')(tower2)

tower3 = MaxPool2D(pool_size = 3 , strides = 1 , padding='same')(input_img)
tower3 = Conv2D(filters=32 , kernel_size=1 , padding = 'same' ,activation = 'relu')(tower3)

In [130]:
concat = concatenate([tower1,tower2,tower3],axis=3,name='concatenation')

In [131]:
print('input shape  : ',input_img.shape)
print('tower1 shape : ',tower1.shape)
print('tower2 shape : ',tower2.shape)
print('tower3 shape : ',tower3.shape)
print('output shape : ',concat.shape)

input shape  :  (None, 28, 28, 1)
tower1 shape :  (None, 28, 28, 32)
tower2 shape :  (None, 28, 28, 32)
tower3 shape :  (None, 28, 28, 32)
output shape :  (None, 28, 28, 96)


In [132]:
output = Flatten()(concat)
print('Output shape after flattening : ',output.shape)

Output shape after flattening :  (None, 75264)


In [133]:
output = Dense(units=10,activation='softmax')(output)

In [134]:
inception = Model(inputs=input_img , outputs=output)

In [135]:
inception.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_74 (Conv2D)              (None, 28, 28, 32)   64          input_18[0][0]                   
__________________________________________________________________________________________________
conv2d_76 (Conv2D)              (None, 28, 28, 32)   64          input_18[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_31 (MaxPooling2D) (None, 28, 28, 1)    0           input_18[0][0]                   
______________________________________________________________________________________________

In [136]:
inception.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

In [138]:
inception.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=32,epochs=2)

Epoch 1/2
1875/1875 [==============================] - 374s 199ms/step - loss: 0.1279 - acc: 0.9615 - val_loss: 0.0583 - val_acc: 0.9817
Epoch 2/2
1875/1875 [==============================] - 370s 197ms/step - loss: 0.0478 - acc: 0.9852 - val_loss: 0.0501 - val_acc: 0.9842
